## Ejercicio 8

Utilizando el archivo **Iris.csv** que contiene información referida a la longitud y al ancho de sépalos y pétalos de tres especies de flores: *iris setosa*, *iris versicolor* e *iris virginica*.  

### a)

Entrenar una multiperceptrón que aprenda a clasificar las 3 clases de flores.  


In [12]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Dense, Input


DATOS_DIR = '../../Datos/'

data = pd.read_csv(DATOS_DIR + 'Iris.csv')
binarizer = preprocessing.LabelBinarizer()

X_raw = np.array(data.iloc[:,:-1])
Y_raw = np.array(data.iloc[:,-1])

Y_raw = binarizer.fit_transform(Y_raw)
scaler = preprocessing.StandardScaler()
X_raw = scaler.fit_transform(X_raw)


ENTRADAS = X_raw.shape[1]
OCULTAS = 2
SALIDAS  = Y_raw.shape[1]
ACTIVACION = 'tanh'

EPOCHS = 500
BATCH_SIZE = 32
ALPHA = 0.02

modelo = Sequential()
modelo.add(Input(shape=(ENTRADAS,)))
modelo.add(Dense(OCULTAS, activation=ACTIVACION))
modelo.add(Dense(SALIDAS, activation='softmax'))


optimizador = optimizers.SGD(learning_rate=ALPHA)
modelo.compile(optimizer=optimizador, loss='categorical_crossentropy', metrics = ['accuracy'])
history = modelo.fit(x=X_raw, y=Y_raw, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4467 - loss: 1.1593  
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4667 - loss: 1.1359 
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4667 - loss: 1.1141 
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4600 - loss: 1.0906 
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4667 - loss: 1.0686 
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4667 - loss: 1.0467 
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4800 - loss: 1.0248 
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4800 - loss: 1.0040 
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4733 - loss: 0.9831 
Epoch 10/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4800 - loss: 0.9630 
Epoch 11/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4733 - loss: 0.9430 
Epoch 12/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4733 - lo

### b)

Utilice Python para calcular la matriz de confusión y calcule de forma manual las métricas de **precision**, **recall**, **accuracy** y **f1-score**. Luego utilice la función **classification_report** de SciKit-Learn para comparar los resultados.

In [ ]:
def evaluar_red(conf_matrix: np.ndarray):
    """
    Dada una matriz de confusión NxN, calcula y muestra:
    - Número total de ejemplos.
    - Número de clases.
    - Accuracy total.
    - Precisión (precision) por clase.
    - Sensibilidad (recall) por clase.
    - F1-score por clase.
    """
    # Total de ejemplos y clases
    total_ejemplos = conf_matrix.sum()
    num_clases = conf_matrix.shape[0]

    # Accuracy total
    correctos = np.trace(conf_matrix)
    accuracy_total = correctos / total_ejemplos

    # Inicialización de métricas
    precision = []
    recall = []
    f1_scores = []

    for i in range(num_clases):
        tp = conf_matrix[i, i]
        fp = conf_matrix[:, i].sum() - tp
        fn = conf_matrix[i, :].sum() - tp

        prec_i = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        rec_i = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1_i = 2 * prec_i * rec_i / (prec_i + rec_i) if (prec_i + rec_i) > 0 else 0.0

        precision.append(prec_i)
        recall.append(rec_i)
        f1_scores.append(f1_i)

    # --- Print de resultados ---
    print(f"Ejemplos en el entrenamiento: {total_ejemplos}")
    print(f"Número de clases: {num_clases}")
    print(f"Accuracy total: {accuracy_total:}\n")

    for i in range(num_clases):
        print(f"Clase {i+1}:")
        print(f"  Precision: {precision[i]:}")
        print(f"  Recall:    {recall[i]:}")
        print(f"  F1-score:  {f1_scores[i]:}")
        print()
        

Y_pred = modelo.predict(X_raw)
Y_it = binarizer.inverse_transform(Y_raw)
Y_pred_it = binarizer.inverse_transform(Y_pred)

report = metrics.classification_report(Y_it, Y_pred_it)
print("Reporte SkLearn:\n%s" % report)
cm = metrics.confusion_matrix(Y_it, Y_pred_it)
print("Confusion matrix:\n%s" % cm)

evaluar_red(metrics.confusion_matrix(np.argmax(Y_raw, axis=1), np.argmax(modelo.predict(X_raw), axis=1)))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Reporte SkLearn:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        50
Iris-versicolor       0.98      0.94      0.96        50
 Iris-virginica       0.94      0.98      0.96        50

       accuracy                           0.97       150
      macro avg       0.97      0.97      0.97       150
   weighted avg       0.97      0.97      0.97       150

Confusion matrix:
[[50  0  0]
 [ 0 47  3]
 [ 0  1 49]]
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Ejemplos en el entrenamiento: 150
Número de clases: 3
Accuracy total: 0.9733333333333334

Clase 1:
  Precision: 1.0
  Recall:    1.0
  F1-score:  1.0

Clase 2:
  Precision: 0.9791666666666666
  Recall:    0.94
  F1-score:  0.9591836734693877

Clase 3:
  Precision: 0.9423076923076923
  Recall:    0.98
  F1-score:  0.9607843137254902

